<a href="https://colab.research.google.com/github/1337Rinz/img_retrieval_vectorDB/blob/main/convert_URLimg_to_Vect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y grpcio pymilvus  # Uninstall existing versions, if any
!pip install grpcio==1.63.0 pymilvus

Found existing installation: grpcio 1.64.1
Uninstalling grpcio-1.64.1:
  Successfully uninstalled grpcio-1.64.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
import os
os.kill(os.getpid(), 9)  # This will restart the kernel

In [1]:
!pip show grpcio pymilvus

Name: grpcio
Version: 1.63.0
Summary: HTTP/2-based RPC framework
Home-page: https://grpc.io
Author: The gRPC Authors
Author-email: grpc-io@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: google-cloud-pubsub, grpc-google-iam-v1, grpcio-status, pymilvus, tensorboard, tensorflow
---
Name: pymilvus
Version: 2.4.4
Summary: Python Sdk for Milvus
Home-page: 
Author: 
Author-email: Milvus Team <milvus-team@zilliz.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: environs, grpcio, milvus-lite, pandas, protobuf, setuptools, ujson
Required-by: 


In [2]:
import cv2
import numpy as np
import requests
from pymilvus import MilvusClient
import json

In [3]:
def get_array(url: str) -> np.ndarray:
    '''
    No preprocess, reshape, resize,...
    '''
    response = requests.get(url)
    image_array = np.asarray(bytearray(response.content), dtype="uint8")
    # Đọc ảnh bằng OpenCV
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    return img


def connect_vector_db(key_dir):
    # Đọc tệp JSON
    with open(key_dir, 'r') as file:
        config = json.load(file)

    cluster_endpoint = config['CLUSTER_ENDPOINT']
    token = config['TOKEN']
    client = MilvusClient(uri=cluster_endpoint, token=token)
    return client


def get_describe_db(client, collections):
    return client.describe_collection(collections)


def insert_vector_db(client, collection, data):
    return client.insert(collection, data=data)


def delete_all_db(client, collection):
    return client.delete(collection_name=collection, filter='Auto_id >= 0')

In [4]:
from transformers import TFViTModel, AutoImageProcessor
import numpy as np
from io import BytesIO
import cv2
import tensorflow as tf
import pandas as pd

In [5]:
image_processor = AutoImageProcessor.from_pretrained(
    "google/vit-base-patch16-224-in21k")
vit_model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkp

In [11]:
filepath = "/content/dta.xlsx"

In [12]:
# -----------ĐỌC FILE EXCEL-------------
df = pd.read_excel(filepath)
url_list = df['URL'].to_list()
describe_list = df['Describe'].to_list()

In [13]:
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bar
import cv2
import requests

# -----------LẤY ẢNH TỪ URL v1.2--------------
tensor = []
for url in tqdm(url_list, desc="Processing Images"):
    # Fetch image from URL using custom function
    img = get_array(url)
    process = image_processor(img, return_tensors="np")['pixel_values']
    tensor.append(process[0])


tensor = np.array(tensor)
print(tensor.shape)
print('Get image: DONE')


Processing Images: 100%|██████████| 6/6 [00:02<00:00,  2.37it/s]

(6, 3, 224, 224)
Get image: DONE


In [14]:
# ------------CALCULATE VECTOR------------
n_samples = tensor.shape[0]
predictions = []
batch_size = 32

for start in tqdm(range(0, n_samples, batch_size), desc="Calculating Vectors"):
    end = min(start + batch_size, n_samples)
    batch_predictions = vit_model(tensor[start:end])['pooler_output'].numpy()
    predictions.append(batch_predictions)

# Stack predictions into a single numpy array
predictions = np.vstack(predictions)
# (n, 768)
print("VECTOR CALCULATED")

Calculating Vectors: 100%|██████████| 1/1 [00:05<00:00,  5.06s/it]

VECTOR CALCULATED


In [ ]:
# --------------INSERT TO DB-----------------
insert_data = []
for vector, url, describe in zip(predictions, url_list, describe_list):
    insert_data.append({
        'vector': vector.tolist(),
        'url': url,
        'describe': describe
    })

client = connect_vector_db('secret_key.json')
result = insert_vector_db(client, 'ViT', insert_data)
client.close()
print("INSERTED TO DB")